# Three-grid P-multigrid Example

In [ ]:
# dependencies
using Pkg
Pkg.activate(".")
Pkg.instantiate()
Pkg.develop(path="../..")
using Plots
using LFAToolkit
using LinearAlgebra

In [ ]:
# setup
finep = 4
midp = 2
coarsep = 1
numbercomponents = 1
dimension = 1

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

mtofbasis = TensorH1LagrangePProlongationBasis(midp + 1, finep + 1, numbercomponents, dimension)
ctombasis = TensorH1LagrangePProlongationBasis(coarsep + 1, midp + 1, numbercomponents, dimension)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep + 1, finep + 1, mesh)
middiffusion = GalleryOperator("diffusion", midp + 1, finep + 1, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep + 1, finep + 1, mesh)

# Chebyshev smoother
finechebyshev = Chebyshev(finediffusion)
midchebyshev = Chebyshev(middiffusion)

# p-multigrid preconditioner
midmultigrid = PMultigrid(middiffusion, coarsediffusion, midchebyshev, [ctombasis])
multigrid = PMultigrid(finediffusion, midmultigrid, finechebyshev, [mtofbasis])

In [ ]:
# full operator symbols
numbersteps = 100
θ_min = -π/2
θ_max = 3π/2
θ_range = LinRange(θ_min, θ_max, numbersteps)

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    ω = [2]
    v = [1, 1]
    maxeigenvalues = zeros(numbersteps)

    # compute
    for i in 1:numbersteps
        θ = [θ_range[i]]
        if abs(θ[1]) >  π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i] = maximum(eigenvalues)
        end
    end

    # plot
    println("max eigenvalue: ", maximum(maxeigenvalues))
    xrange = θ_range/π
    plot(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        maxeigenvalues,
        ytickfont=font(12, "Courier"),
        ylabel="spectral radius",
        linewidth=3,
        legend=:none,
        title="P-Multigrid Error Symbol"
    )
    ylims!(0.0, maximum(maxeigenvalues) * 1.1)
# -- 2D --
elseif dimension == 2
    # setup
    ω = [2]
    v = [1, 1]
    maxeigenvalues = zeros(numbersteps, numbersteps)

    # compute
    for i in 1:numbersteps, j in 1:numbersteps
        θ = [θ_range[i], θ_range[j]]
        if sqrt(abs(θ[1])^2 + abs(θ[2])^2) > π/128
            A = computesymbols(multigrid, ω, v, θ)
            eigenvalues = [abs(val) for val in eigvals(A)]
            maxeigenvalues[i, j] = maximum(eigenvalues)
        end
    end

    # plot
    println("max eigenvalue: ", maximum(maxeigenvalues))
    xrange = θ_range/π
    heatmap(
        xrange,
        xlabel="θ/π",
        xtickfont=font(12, "Courier"),
        xrange,
        ylabel="θ/π",
        ytickfont=font(12, "Courier"),
        maxeigenvalues,
        title="P-Multigrid Error Symbol",
        transpose=true,
        aspect_ratio=:equal
    )
    xlims!(θ_min/π, θ_max/π)
    ylims!(θ_min/π, θ_max/π)
end